# Vertex AI Search - Generic Search Filter Demo

A demo of how to filter search results 

Following this official example here: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/search_filters_metadata.ipynb


Refs 

* https://cloud.google.com/generative-ai-app-builder/docs/filter-search-metadata


## Summary of steps


* obtain sample data files (PDFs)
* define users and access rules for sample data files
* map rules to filter criteria for search 
* run filtered search query

## Objective

This notebook shows how to use [filters and metadata](https://cloud.google.com/generative-ai-app-builder/docs/filter-search-metadata) in search requests to [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/introduction).

This works with unstructrued apps that contain metadata. You can use metadata fields to restrict your search to a specific set of documents.

Services used in the notebook:

*   ✅ Vertex AI Search for document search and retrieval


## Installs 


In [ ]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.discoveryengine','google-cloud-discoveryengine')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.


In [ ]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Setup
inputs:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
print(PROJECT_ID)

In [ ]:
from google.cloud import storage

import json

from google.cloud import discoveryengine_v1alpha as discoveryengine
from google.api_core.client_options import ClientOptions

In [ ]:
# PROJECT_ID = '' # set above
REGION = 'us-central1'
EXPERIMENT = 'search-alphabet-investor-pdfs'
SERIES = "generative-ai"

LOCATION="global"

In [ ]:
BUCKET = SERIES + EXPERIMENT 
BUCKET_URI = f"gs://{BUCKET}"

In [ ]:
LOCATION = "global"  # Replace with your data store location
# The datastore name can only contain lowercase letters, numbers, and hyphens
# DATASTORE_NAME = EXPERIMENT
DATASTORE_NAME = "test-data-store"
DATASTORE_ID = f"{DATASTORE_NAME}-id"

## Create GCS Bucket 

In [ ]:
gcs = storage.Client(project = PROJECT_ID)

In [ ]:
if not gcs.lookup_bucket(BUCKET):
    print("Bucket does not exist, creating it now...")
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(bucket)
else:
    print("Bucket already exists:")
    print(gcs.lookup_bucket(BUCKET))

## Ingest sample data

In [ ]:
# ! gsutil -m cp gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/* $BUCKET_URI # TODO - all pdfs 
! gsutil cp gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/20040630_google_10Q.pdf $BUCKET_URI
! gsutil cp gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/20040930_google_10Q.pdf $BUCKET_URI

## Create metadata file


In [ ]:
# TODO - update to be correct sample
# metadata = [
#     {
#    "id": "",
#    "jsonData": "",
#    "content": {
#      "mimeType": "<application/pdf>",
#      "uri": "gs://generative-aisearch-alphabet-investor-pdfs/20040630_google_10Q.pdf"
#    },
#    "acl_info": {
#      "readers": [
#        {
#          "principals": [
#            { "user_id": "bruce@justinjm.altostrat.com"}
#          ]
#        }
#      ]
#    }
#     },
#      {
#    "id": "",
#    "jsonData": "",
#    "content": {
#      "mimeType": "<application/pdf>",
#      "uri": "gs://generative-aisearch-alphabet-investor-pdfs/20040930_google_10Q.pdf"
#    },
#    "acl_info": {
#      "readers": [
#        {
#          "principals": [
#            { "user_id": "admin@justinjm.altostrat.com"},
#          ]
#        }
#      ]
#    }
#  }
    
    
# ]
   


### Write metadata  to a local file

In [ ]:
# metadata_filename = "metadata.jsonl"

# # Write to a .jsonl file
# with open(metadata_filename,  'w') as file:
#     for item in metadata:
#         json_string = json.dumps(item)
#         file.write(json_string + '\n')

### Upload metadata to cloud storage

In [ ]:
# ! gsutil -m cp $metadata_filename $BUCKET_URI/$metadata_filename

## Create Datastore

In [ ]:
def create_data_store(
    project_id: str, location: str, data_store_name: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DataStoreServiceClient(client_options=client_options)

    # Initialize request argument(s)
    data_store = discoveryengine.DataStore(
        display_name=data_store_name,
        industry_vertical="GENERIC",
        content_config="CONTENT_REQUIRED"
    )

    request = discoveryengine.CreateDataStoreRequest(
        parent=discoveryengine.DataStoreServiceClient.collection_path(
            project_id, location, "default_collection"
        ),
        data_store=data_store,
        data_store_id=data_store_id,
    )
    operation = client.create_data_store(request=request)

    # Make the request
    # The try block is necessary to prevent execution from haulting due to an error being thrown when the datastore takes a while to instantiate
    try:
        response = operation.result(timeout=90)
    except:
        print("long-running operation")

In [ ]:
create_data_store(PROJECT_ID, LOCATION, DATASTORE_NAME, DATASTORE_ID)

## Import documents to datastore

In [ ]:
# def import_documents(
#     project_id: str,
#     location: str,
#     data_store_id: str,
#     gcs_uri: str,
# ):
#     # Create a client
#     client_options = (
#         ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
#         if location != "global"
#         else None
#     )
#     client = discoveryengine.DocumentServiceClient(client_options=client_options)

#     # The full resource name of the search engine branch.
#     # e.g. projects/{project}/locations/{location}/dataStores/{data_store_id}/branches/{branch}
#     parent = client.branch_path(
#         project=project_id,
#         location=location,
#         data_store=data_store_id,
#         branch="default_branch",
#     )

#     source_documents = [f"{gcs_uri}/*"]

#     request = discoveryengine.ImportDocumentsRequest(
#         parent=parent,
#         gcs_source=discoveryengine.GcsSource(
#             input_uris=source_documents, data_schema="content"
#         ),
#         # Options: `FULL`, `INCREMENTAL`
#         reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
#     )

#     # Make the request
#     operation = client.import_documents(request=request)

#     response = operation.result()

#     # Once the operation is complete,
#     # get information from operation metadata
#     metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

#     # Handle the response
#     return operation.operation.name

In [ ]:
# import_documents(PROJECT_ID, LOCATION, DATASTORE_ID, BUCKET_URI)

## query documents

### Helper function

In [ ]:
# def search_data_store(
#     project_id: str,
#     location: str,
#     data_store_id: str,
#     search_query: str,
#     filter_str: str,
# ) -> discoveryengine.SearchResponse:
#     #  For more information, refer to:
#     # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
#     client_options = (
#         ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
#         if location != "global"
#         else None
#     )

#     # Create a client
#     client = discoveryengine.SearchServiceClient(client_options=client_options)

#     # The full resource name of the search engine serving config
#     # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
#     serving_config = client.serving_config_path(
#         project=project_id,
#         location=location,
#         data_store=data_store_id,
#         serving_config="default_config",
#     )

#     # Optional: Configuration options for search
#     # Refer to the `ContentSearchSpec` reference for all supported fields:
#     # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
#     content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
#         # For information about snippets, refer to:
#         # https://cloud.google.com/generative-ai-app-builder/docs/snippets
#         snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
#             return_snippet=True
#         ),
#         extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
#             max_extractive_answer_count=5,
#             max_extractive_segment_count=1,
#         ),
#         # For information about search summaries, refer to:
#         # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
#         summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
#             summary_result_count=5,
#             include_citations=True,
#             ignore_adversarial_query=False,
#             ignore_non_summary_seeking_query=False,
#         ),
#     )

#     # Refer to the `SearchRequest` reference for all supported fields:
#     # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
#     request = discoveryengine.SearchRequest(
#         serving_config=serving_config,
#         query=search_query,
#         filter=filter_str,
#         page_size=5,
#         content_search_spec=content_search_spec,
#         query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
#             condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
#         ),
#         spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
#             mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
#         ),
#     )

#     response = client.search(request)
#     return response

### Example 1 - simple

In [ ]:
# search_query = "how to process a claim"
# filter_str = 'name: ANY("Document1")'

# results = search_data_store(
#     PROJECT_ID, LOCATION, DATA_STORE_ID, search_query, filter_str
# )

# print(f"\nQuestion: '{search_query}'\n\n")
# print("Summary" + "-" * 40)
# print(results.summary.summary_text)

# print("Raw Results" + "-" * 40)
# print(results)

### Example 2 - more complex

In [ ]:
# search_query = "how to process a claim"
# filter_str = 'name: ANY("Document1") AND category: ANY("PersonaA")'

# results = search_data_store(
#     PROJECT_ID, LOCATION, DATA_STORE_ID, search_query, filter_str
# )

# print(f"\nQuestion: '{search_query}'\n\n")
# print("Summary" + "-" * 40)
# print(results.summary.summary_text)

# print("Raw Results" + "-" * 40)
# print(results)

In [ ]:
## Inp

In [ ]:
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1beta as discoveryengine


def search_data_store(
    project_id: str,
    location: str,
    data_store_id: str,
    search_query: str,
    filter_str: str,
) -> discoveryengine.SearchResponse:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search engine serving config
    # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
    serving_config = client.serving_config_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        serving_config="default_config",
    )

    # Optional: Configuration options for search
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_answer_count=5,
            max_extractive_segment_count=1,
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=False,
            ignore_non_summary_seeking_query=False,
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        filter=filter_str,
        page_size=5,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    return response